In [1]:
#original data size = 512x712x34 

# fd_class1 = np.random.randint(low=30,high=60,size=(128,128,128),dtype='int')
# fd_class2 = np.random.randint(low=0,high=29,size=(128,128,128),dtype='int')
# y1 = 1
# y2 = 0
# fd_class1 = np.expand_dims(fd_class1, axis=0)
# fd_class2 = np.expand_dims(fd_class2, axis=0)
# loader = [(fd_class1, y1), (fd_class2,y2)]


#train = np.ndarray.flatten(fd_class1)
#train2 = np.ndarray.flatten(fd_class2)

# train_dataset, validation_dataset, test_dataset = random_split(
#     full_dataset,
#     [num_training, num_validation, num_test],
#     generator=torch.Generator().manual_seed(23061912))

#dataloader = DataLoader(train_dataset, batch_size=8, drop_last=True, sampler=sampler)

# Other VGG Classes

In [52]:
# class VGG(nn.Module):

#     def __init__(self, features, num_classes=1000, init_weights=True):
#         super(VGG, self).__init__()
#         self.features = features
#         self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        
#         self.classifier = nn.Sequential(
#             nn.Linear(512 * 7 * 7, 4096),
#             nn.ReLU(True),
#             nn.Dropout(),
#             nn.Linear(4096, 4096),
#             nn.ReLU(True),
#             nn.Dropout(),
#             nn.Linear(4096, num_classes),
#         )
        
#         if init_weights:
#             self._initialize_weights()

#     def forward(self, x):
#         x = self.features(x)
#         x = self.avgpool(x)
#         x = torch.flatten(x, 1)
#         x = self.classifier(x)
#         return x

#     def _initialize_weights(self):
#         for m in self.modules():
#             if isinstance(m, nn.Conv2d):
#                 nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
#                 if m.bias is not None:
#                     nn.init.constant_(m.bias, 0)
#             elif isinstance(m, nn.BatchNorm2d):
#                 nn.init.constant_(m.weight, 1)
#                 nn.init.constant_(m.bias, 0)
#             elif isinstance(m, nn.Linear):
#                 nn.init.normal_(m.weight, 0, 0.01)
#                 nn.init.constant_(m.bias, 0)

In [ ]:
# class Vgg2D(torch.nn.Module):

#     def __init__(
#             self,
#             input_size,
#             fmaps=12,
#             downsample_factors=[(2, 2), (2, 2), (2, 2), (2, 2)],
#             output_classes=6):

#         super(Vgg2D, self).__init__()

#         self.input_size = input_size

#         current_fmaps, h, w = tuple(input_size)
#         current_size = (h, w)

#         features = []
#         for i in range(len(downsample_factors)):

#             features += [
#                 torch.nn.Conv2d(
#                     current_fmaps,
#                     fmaps,
#                     kernel_size=3,
#                     padding=1),
#                 torch.nn.BatchNorm2d(fmaps),
#                 torch.nn.ReLU(inplace=True),
#                 torch.nn.Conv2d(
#                     fmaps,
#                     fmaps,
#                     kernel_size=3,
#                     padding=1),
#                 torch.nn.BatchNorm2d(fmaps),
#                 torch.nn.ReLU(inplace=True),
#                 torch.nn.MaxPool2d(downsample_factors[i])
#             ]

#             current_fmaps = fmaps
#             fmaps *= 2

#             size = tuple(
#                 int(c/d)
#                 for c, d in zip(current_size, downsample_factors[i]))
#             check = (
#                 s*d == c
#                 for s, d, c in zip(size, downsample_factors[i], current_size))
#             assert all(check), \
#                 "Can not downsample %s by chosen downsample factor" % \
#                 (current_size,)
#             current_size = size

#         self.features = torch.nn.Sequential(*features)

#         classifier = [
#             torch.nn.Linear(
#                 current_size[0] *
#                 current_size[1] *
#                 current_fmaps,
#                 4096),
#             torch.nn.ReLU(inplace=True),
#             torch.nn.Dropout(),
#             torch.nn.Linear(
#                 4096,
#                 4096),
#             torch.nn.ReLU(inplace=True),
#             torch.nn.Dropout(),
#             torch.nn.Linear(
#                 4096,
#                 output_classes)
#         ]

#         self.classifier = torch.nn.Sequential(*classifier)
    
#     def forward(self, raw):

#         # add a channel dimension to raw
#         # shape = tuple(raw.shape)
#         # raw = raw.reshape(shape[0], 1, shape[1], shape[2])
        
#         # compute features
#         f = self.features(raw)
#         f = f.view(f.size(0), -1)
        
#         # classify
#         y = self.classifier(f)

#         return y

# Other Training frameworks

In [ ]:
#Training on fake data
def train(model, loss_function):
    if torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")

    model.train()
    epoch_loss = 0
    num_batches = 0
    model = model.to(device)
    
    for batch_id, (x, y) in enumerate(loader):
        # move input and target to the active device (either cpu or gpu)
        # x = np.stack(x, axis= 0)
        #x, y = x.to(device), y.to(device)
        x = x[np.newaxis]
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
        print(x.shape)
        print(y.dtype)
        x, y = x.to(device), y.to(device)
        # zero the gradients for this iteration
        optimizer.zero_grad()

        # apply model and calculate loss
        prediction = model(x)
        loss = loss_function(prediction, y)

        # backpropagate the loss and adjust the parameters
        loss.backward()
        optimizer.step()

        # # log to console
        # if batch_id % log_interval == 0:
        #     print(
        #         "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
        #             epoch,
        #             batch_id * len(x),
        #             len(loader.dataset),
        #             100.0 * batch_id / len(loader),
        #             loss.item(),
        #         )
        #     )
    

In [112]:
from tqdm import tqdm

def train():
    if torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")
        
    loss=[] 
    counter=[]
    for epoch in tqdm(range(epochs)):
        epoch_loss = 0
        for i, (x, y, isMatch) in enumerate(loader):
            
            # x = torch.from_numpy(x).to(device).float()
            # y = torch.from_numpy(np.array([y])).to(device).float()
            #print(type(vol1))
            img1 = torch.from_numpy(vol1).float()
            img2 = torch.from_numpy(vol2).float()
            y = torch.from_numpy(np.array([y])).float()

            #vol1, vol2 
            img1, img2 , y = img1.to(device), img2.to(device), y.to(device)

            optimizer.zero_grad()

            output1 = model(img1)
            output2 = model(img2)

            isMatch = torch.from_numpy(np.array([isMatch])).float() 
            isMatch = isMatch.to(device)

            loss = loss_function(output1, output2, isMatch)

            loss.backward()
            optimizer.step()    
            epoch_loss += loss
            
        print(f"epoch {epoch}, training loss={epoch_loss}")
    
    return model

def validate():    
    model.eval()
    #dataloader = DataLoader(validation_dataset, batch_size=32)
    
    return evaluate(dataloader, 'validate')

def test():    
    model.eval()
    dataloader = DataLoader(test_dataset, batch_size=32)
    
    return evaluate(dataloader, 'test')

print(type(vol1))
#model = train(vol1, vol2, y)
model = train()

<class 'numpy.ndarray'>


  0%|          | 1/2000 [00:00<06:30,  5.12it/s]

epoch 0, training loss=0.0008545517921447754


  0%|          | 2/2000 [00:00<05:02,  6.60it/s]

epoch 1, training loss=0.0012387633323669434


...

epoch 1997, training loss=7.408857345581055e-05


100%|█████████▉| 1999/2000 [03:50<00:00,  8.62it/s]

epoch 1998, training loss=6.622076034545898e-05


100%|██████████| 2000/2000 [03:50<00:00,  8.69it/s]

epoch 1999, training loss=6.61611557006836e-05
